Atividade #11: Indo Além (Hugging Face)

Integrantes: Jorge Mutran, kaick Nishiya, Lucas koiti, Lucas Prado, Lucas Quinto e Matheus Pestilli

Passo 1: Tópico Escolhido
Tópico: 1 RAG (Retrieved Augmented Generation)

Passo 2: Construção da Aplicação (Código Python)
Vamos criar um "Chatbot Especialista em Filmes". Ele terá uma pequena base de conhecimento (que o modelo original não conhece necessariamente em detalhes) e usará RAG para responder.

Instalação das bibliotecas

In [2]:
# Instalando as bibliotecas necessárias do ecossistema Hugging Face e vetores
!pip install -q transformers sentence-transformers faiss-cpu

 O Código da Aplicação RAG

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# --- 1. Base de Conhecimento (Simulando um banco de dados) ---
# Texto fictício ou específico que o modelo deve consultar
documents = [
    "O filme 'A Jornada do Dev' foi lançado em 2024 e conta a história de um programador que descobriu um bug na Matrix.",
    "A linguagem de programação Python foi criada por Guido van Rossum e lançada em 1991.",
    "Para instalar bibliotecas no Python, utilizamos o gerenciador de pacotes chamado pip.",
    "O framework Hugging Face facilita o uso de modelos de Deep Learning para processamento de linguagem natural."
]

# --- 2. Indexação (Criando os Vetores) ---
print("Criando embeddings...")
# Usamos um modelo leve e eficiente para transformar texto em números (embeddings)
encoder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = encoder.encode(documents)

# Criando o índice FAISS (Banco de dados vetorial em memória)
d = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(doc_embeddings)

# --- 3. Função de Busca (Retrieval) ---
def search(query, k=1):
    # Transforma a pergunta do usuário em vetor
    query_vector = encoder.encode([query])
    # Busca o vetor mais próximo na base de dados
    distances, indices = index.search(query_vector, k)
    return documents[indices[0][0]]

# --- 4. Geração da Resposta (Generation) ---
# Usamos o FLAN-T5, um modelo excelente do Google disponível no Hugging Face
generator = pipeline('text2text-generation', model='google/flan-t5-base')

def rag_chatbot(user_query):
    print(f"Pergunta: {user_query}")

    # Passo R (Retrieve): Busca a informação relevante
    context = search(user_query)
    print(f"Contexto Encontrado: {context}")

    # Passo G (Generate): Gera a resposta baseada no contexto
    # Prompt Engineering simples
    prompt = f"Context: {context}\n\nQuestion: {user_query}\n\nAnswer based on the context:"

    response = generator(prompt, max_length=100)[0]['generated_text']
    return response

# --- 5. Testando a Aplicação ---
print("-" * 50)
# Pergunta sobre algo que está na nossa base fictícia
resposta = rag_chatbot("Quando foi lançado o filme A Jornada do Dev?")
print(f"Resposta da IA: {resposta}")

print("-" * 50)
# Pergunta técnica baseada na base
resposta_tec = rag_chatbot("Quem criou o Python?")
print(f"Resposta da IA: {resposta_tec}")

Criando embeddings...


Device set to use cpu


--------------------------------------------------
Pergunta: Quando foi lançado o filme A Jornada do Dev?
Contexto Encontrado: O filme 'A Jornada do Dev' foi lançado em 2024 e conta a história de um programador que descobriu um bug na Matrix.


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Resposta da IA: 2024
--------------------------------------------------
Pergunta: Quem criou o Python?
Contexto Encontrado: A linguagem de programação Python foi criada por Guido van Rossum e lançada em 1991.


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Resposta da IA: Guido van Rossum
